### imports

In [1]:
import pandas as pd
import numpy as np
import requests
import os
from datetime import datetime, timedelta

from tqdm import tqdm
import json
import pytz

# HORA MINIMA 2020/01 -> 2023/04
# formatado data final/inicial 2023-03


In [2]:
def toDate24(df,coluna):
    brasilia_timezone = pytz.timezone('America/Sao_Paulo')

    # Converte o timestamp para o fuso horário de Brasília
    nova_coluna = coluna + "_formatted"
    df[nova_coluna] = df[coluna].apply(lambda x: datetime.fromtimestamp(int(x)/1000, tz=brasilia_timezone).strftime("%A, %B %d, %Y %H:%M:%S"))

    return df


### /bimtra

Fazer a requisição necessaria para baixar os dados e salvar em "raw_bimtra"


In [3]:

def toRawBimtra(pathfile,dataInicial,dataFinal):
    def bimtra_req(idate,fdate):
        url = 'http://montreal.icea.decea.mil.br:5002/api/v1/bimtra'
        params = {
            'token': 'a779d04f85c4bf6cfa586d30aaec57c44e9b7173',
            'idate': idate,
            'fdate': fdate
        }
        headers = {
            'accept': 'application/json'
        }

        response = requests.get(url, params=params, headers=headers)

        if response.status_code == 200:
            data = response.json()
            return data
        else:
            print(f"Falha na requisição. Código de status: {response.status_code}")

    dateArray = np.arange(np.datetime64(dataInicial), np.datetime64(dataFinal), np.timedelta64(1,'M'))
    print(dateArray)
    pathfile = '../data/raw_data/bimtra_raw.txt'
    if os.path.exists(pathfile) :
        first = False
    else:
        first = True

    for aux in tqdm(range(len(dateArray)-1)):
        idate = str(dateArray[aux])+"-01"
        fdate = str(dateArray[aux+1])+"-01"
        print(f"\nidate: {idate} , fdate: {fdate}")
        data = bimtra_req(idate,fdate)
        data = str(data).replace("'",'"')
        if str(data) != "[]": 
            with open(pathfile, 'a') as arquivo:
                if first:
                    arquivo.write(str(data)[:-1])
                    first = False
                    
                else:
                    arquivo.write(","+str(data)[1:-1])
                print("Saved data")
    with open(pathfile, 'a') as arquivo:
        arquivo.write("]")

def toCSVBimtra(pathfile):
    with open(pathfile, 'r') as arquivo:
        # Carregue o conteúdo do arquivo em uma lista de dicionários
        data_complete = json.load(arquivo)

    df = pd.DataFrame(data_complete)
    df = toDate24(df,"dt_dep")
    df = toDate24(df,"dt_arr")


    df.to_csv("../data/processed_data/bimtra_bronze.csv",index=False)
    
def ingestaoBimtra(dataInicial, dataFinal):
    pathfile =  '../data/raw_data/bimtra_raw.txt'
    if os.path.exists(pathfile):         
        os.remove(pathfile)
    toRawBimtra(pathfile,dataInicial, dataFinal)
    toCSVBimtra(pathfile)


### /espera

Fazer a requisição necessaria para baixar os dados e salvar em "raw_espera"


In [4]:

def toRawEspera(pathfile,dataInicial,dataFinal):
    def espera_req(idate,fdate):
        url = 'http://montreal.icea.decea.mil.br:5002/api/v1/esperas'
        params = {
            'token': 'a779d04f85c4bf6cfa586d30aaec57c44e9b7173',
            'idate': idate,
            'fdate': fdate
        }
        headers = {
            'accept': 'application/json'
        }
        response = requests.get(url, params=params, headers=headers)
        if response.status_code == 200:
            data = response.json()
            return data
        else:
            print(f"Falha na requisição. Código de status: {response.status_code}")
    dateArray = np.arange(np.datetime64(dataInicial), np.datetime64(dataFinal), np.timedelta64(1,'M'))

    if os.path.exists(pathfile) :
        first = False
    else:
        first = True

    for aux in tqdm(range(len(dateArray)-1)):
        idate = str(dateArray[aux])+"-01"
        fdate = str(dateArray[aux+1])+"-01"
        print(f"\nidate: {idate} , fdate: {fdate}")
        data = espera_req(idate,fdate)
        data = str(data).replace("'",'"')
        if str(data) != "[]": 
            with open(pathfile, 'a') as arquivo:
                if first:
                    arquivo.write(str(data)[:-1])
                    first = False
                else:
                    arquivo.write(","+str(data)[1:-1])
                print("Saved data")
    with open(pathfile, 'a') as arquivo:
        arquivo.write("]")

def toCSVEspera(pathfile):
    with open(pathfile, 'r') as arquivo:
    # Carregue o conteúdo do arquivo em uma lista de dicionários
        data_complete = json.load(arquivo)

    df = pd.DataFrame(data_complete)

    df = toDate24(df,'hora')

    df.to_csv("../data/processed_data/espera_bronze.csv",index=False)

def ingestaoEspera(dataInicial,dataFinal):
    pathfile = '../data/raw_data/espera_raw.txt'
    if os.path.exists(pathfile):         
        os.remove(pathfile)
    toRawEspera(pathfile,dataInicial,dataFinal)
    toCSVEspera(pathfile)



### /MetaF

Fazer a requisição necessaria para baixar os dados e salvar em "raw_metaf"

In [5]:
def toRawMetaF(pathfile,dataInicial,dataFinal): 

    def metaf_req(idate,fdate):
        url = 'http://montreal.icea.decea.mil.br:5002/api/v1/metaf'
        params = {
            'token': 'a779d04f85c4bf6cfa586d30aaec57c44e9b7173',
            'idate': idate,
            'fdate': fdate
        }
        headers = {
            'accept': 'application/json'
        }

        response = requests.get(url, params=params, headers=headers)

        if response.status_code == 200:
            data = response.json()
            return data
        else:
            print(f"Falha na requisição. Código de status: {response.status_code}")

    dateArray = np.arange(np.datetime64(dataInicial), np.datetime64(dataFinal), np.timedelta64(1,'M'))

    if os.path.exists(pathfile) :
        first = False
    else:
        first = True

    for aux in tqdm(range(len(dateArray)-1)):
        idate = str(dateArray[aux])+"-01"
        fdate = str(dateArray[aux+1])+"-01"
        print(f"\nidate: {idate} , fdate: {fdate}")
        data = metaf_req(idate,fdate)
        data = str(data).replace("'",'"')
        if str(data) != "[]": 
            with open(pathfile, 'a') as arquivo:
                if first:
                    arquivo.write(str(data)[:-1])
                    first = False
                    
                else:
                    arquivo.write(","+str(data)[1:-1])
                print("Saved data")
    with open(pathfile, 'a') as arquivo:
        arquivo.write("]")

def toCSVMetaF(pathfile):
    with open(pathfile, 'r') as arquivo:
    # Carregue o conteúdo do arquivo em uma lista de dicionários
        data_complete = json.load(arquivo)

    df = pd.DataFrame(data_complete)

    df = toDate24(df,'hora')

    df.to_csv("../data/processed_data/metaf_bronze.csv",index=False)


def ingestaoMetaf(dataInicial,dataFinal):
    pathfile = '../data/raw_data/metaf_raw.txt'
    if os.path.exists(pathfile):         
        os.remove(pathfile)
    toRawMetaF(pathfile,dataInicial,dataFinal)
    toCSVMetaF(pathfile)



### /MetaR

Fazer a requisição necessaria para baixar os dados e salvar em "raw_metar"

In [6]:
def toRawMetaR(pathfile,dataInicial,dataFinal):
    def metar_req(idate,fdate):
        url = 'http://montreal.icea.decea.mil.br:5002/api/v1/metar'
        params = {
            'token': 'a779d04f85c4bf6cfa586d30aaec57c44e9b7173',
            'idate': idate,
            'fdate': fdate
        }
        headers = {
            'accept': 'application/json'
        }

        response = requests.get(url, params=params, headers=headers)

        if response.status_code == 200:
            data = response.json()
            return data
        else:
            print(f"Falha na requisição. Código de status: {response.status_code}")

    dateArray = np.arange(np.datetime64(dataInicial), np.datetime64(dataFinal), np.timedelta64(1,'M'))
    
    if os.path.exists(pathfile) :
        first = False
    else:
        first = True

    for aux in tqdm(range(len(dateArray)-1)):
        idate = str(dateArray[aux])+"-01"
        fdate = str(dateArray[aux+1])+"-01"
        print(f"\nidate: {idate} , fdate: {fdate}")
        data = metar_req(idate,fdate)
        data = str(data).replace("'",'"')
        if str(data) != "[]": 
            with open(pathfile, 'a') as arquivo:
                if first:
                    arquivo.write(str(data)[:-1])
                    first = False
                    
                else:
                    arquivo.write(","+str(data)[1:-1])
                print("Saved data")
    with open(pathfile, 'a') as arquivo:
        arquivo.write("]")

def toCSVMetar(pathfile):
    with open(pathfile, 'r') as arquivo:
    # Carregue o conteúdo do arquivo em uma lista de dicionários
        data_complete = json.load(arquivo)

    df = pd.DataFrame(data_complete)
    df = toDate24(df,'hora')

    df.to_csv("../data/processed_data/metar_bronze.csv",index=False)

def ingestaoMetaR(dataInicial,dataFinal):
    pathfile = '../data/raw_data/metar_raw.txt'
    if os.path.exists(pathfile):         
        os.remove(pathfile)
    toRawMetaR(pathfile,dataInicial,dataFinal)
    toCSVMetar(pathfile)


### /Tc-Prev


Fazer a requisição necessaria para baixar os dados e salvar em "raw_tc-prev"

In [7]:
def toRawTcPrev(pathfile,dataInicial,dataFinal):
    def metaf_req(idate,fdate):
        url = 'http://montreal.icea.decea.mil.br:5002/api/v1/tc-prev'
        params = {
            'token': 'a779d04f85c4bf6cfa586d30aaec57c44e9b7173',
            'idate': idate,
            'fdate': fdate
        }
        headers = {
            'accept': 'application/json'
        }

        response = requests.get(url, params=params, headers=headers)

        if response.status_code == 200:
            data = response.json()
            return data
        else:
            print(f"Falha na requisição. Código de status: {response.status_code}")

    dateArray = np.arange(np.datetime64(dataInicial), np.datetime64(dataFinal), np.timedelta64(1,'M'))

    if os.path.exists(pathfile) :
        first = False
    else:
        first = True

    for aux in tqdm(range(len(dateArray)-1)):
        idate = str(dateArray[aux])+"-01"
        fdate = str(dateArray[aux+1])+"-01"
        print(f"\nidate: {idate} , fdate: {fdate}")
        data = metaf_req(idate,fdate)
        data = str(data).replace("'",'"')
        if str(data) != "[]": 
            with open(pathfile, 'a') as arquivo:
                if first:
                    arquivo.write(str(data)[:-1])
                    first = False
                    
                else:
                    arquivo.write(","+str(data)[1:-1])
                print("Saved data")
    with open(pathfile, 'a') as arquivo:
        arquivo.write("]")

def toCSVTcPrev(pathfile):
    with open(pathfile, 'r') as arquivo:
    # Carregue o conteúdo do arquivo em uma lista de dicionários
        data_complete = json.load(arquivo)

    df = pd.DataFrame(data_complete)

    df = toDate24(df,'hora')
    df.to_csv("../data/processed_data/tc-prev_bronze.csv",index=False)

def ingestaoTcPrev(dataInicial,dataFinal):
    pathfile = '../data/raw_data/tc-prev_raw.txt'
    if os.path.exists(pathfile):         
        os.remove(pathfile)
    toRawTcPrev(pathfile,dataInicial,dataFinal)
    toCSVTcPrev(pathfile)


### /Tc-real


Fazer a requisição necessaria para baixar os dados e salvar em "raw_tc-real"

In [8]:
def toRawTcreal(pathfile,dataInicial,dataFinal):
    def metaf_req(idate,fdate):
        url = 'http://montreal.icea.decea.mil.br:5002/api/v1/tc-real'
        params = {
            'token': 'a779d04f85c4bf6cfa586d30aaec57c44e9b7173',
            'idate': idate,
            'fdate': fdate
        }
        headers = {
            'accept': 'application/json'
        }

        response = requests.get(url, params=params, headers=headers)

        if response.status_code == 200:
            data = response.json()
            return data
        else:
            print(f"Falha na requisição. Código de status: {response.status_code}")

    dateArray = np.arange(np.datetime64(dataInicial), np.datetime64(dataFinal), np.timedelta64(1,'M'))
    

    if os.path.exists(pathfile) :
        first = False
    else:
        first = True

    for aux in tqdm(range(len(dateArray)-1)):
        idate = str(dateArray[aux])+"-01"
        fdate = str(dateArray[aux+1])+"-01"
        print(f"\nidate: {idate} , fdate: {fdate}")
        data = metaf_req(idate,fdate)
        data = str(data).replace("'",'"')
        if str(data) != "[]": 
            with open(pathfile, 'a') as arquivo:
                if first:
                    arquivo.write(str(data)[:-1])
                    first = False
                    
                else:
                    arquivo.write(","+str(data)[1:-1])
                print("Saved data")
    with open(pathfile, 'a') as arquivo:
        arquivo.write("]")

def toCSVTcReal(pathfile):
    with open(pathfile, 'r') as arquivo:
    # Carregue o conteúdo do arquivo em uma lista de dicionários
        data_complete = json.load(arquivo)

    df = pd.DataFrame(data_complete)

    df = toDate24(df,'hora')

    df.to_csv("../data/processed_data/tc-real_bronze.csv",index=False)

def ingestaoTcReal(dataInicial,dataFinal):
    pathfile = '../data/raw_data/tc-real_raw.txt'
    if os.path.exists(pathfile):         
        os.remove(pathfile)
    toRawTcreal(pathfile,dataInicial,dataFinal)
    toCSVTcReal(pathfile)



### /cat-62


Fazer a requisição necessaria para baixar os dados e salvar em "raw_cat-62"

In [9]:
def toRawCat62(pathfile,dataInicial,dataFinal):
    def cat62_req(idate,fdate):
        url = 'http://montreal.icea.decea.mil.br:5002/api/v1/cat-62'
        params = {
            'token': 'a779d04f85c4bf6cfa586d30aaec57c44e9b7173',
            'idate': idate,
            'fdate': fdate
        }
        headers = {
            'accept': 'application/json'
        }

        response = requests.get(url, params=params, headers=headers)

        if response.status_code == 200:
            data = response.json()
            return data
        else:
            print(f"Falha na requisição. Código de status: {response.status_code}")
    # Adicione o tempo correto nas datas
    dataInicial = dataInicial + '-01 00:00:00.000'
    dataFinal = dataFinal + '-01 00:00:00.000'

    # Converta as datas para objetos np.datetime64
    dataInicial_np = np.datetime64(dataInicial)
    dataFinal_np = np.datetime64(dataFinal)

    # Crie um array de datas em intervalos de 1 hora
    dateArray = np.arange(dataInicial_np, dataFinal_np, np.timedelta64(1, 'h'))

    if os.path.exists(pathfile) :
        first = False
    else:
        first = True

    for aux in tqdm(range(len(dateArray)-1)):
        idate = str(dateArray[aux]).replace("T"," ")
        fdate = str(dateArray[aux+1]).replace("T"," ")
        print(f"\nidate: {idate} , fdate: {fdate}")
        data = cat62_req(idate,fdate)
        data = str(data).replace("'",'"')
        if str(data) != "[]": 
            with open(pathfile, 'a') as arquivo:
                if first:
                    data = data.replace("None","0.0")
                    arquivo.write(str(data)[:-1])
                    first = False
                    
                else:
                    data = data.replace("None","0.0")
                    arquivo.write(","+str(data)[1:-1])
                print("Saved data")
    with open(pathfile, 'a') as arquivo:
        arquivo.write("]")

def toCSVCat62(pathfile):
    with open(pathfile, 'r') as arquivo:
    # Carregue o conteúdo do arquivo em uma lista de dicionários
        data_complete = json.load(arquivo)

    df = pd.DataFrame(data_complete)

    df = toDate24(df,'dt_radar')

    df.to_csv("../data/processed_data/cat62_bronze.csv",index=False)

def ingestaoCat62(dataInicial,dataFinal):
    pathfile = '../data/raw_data/cat62_raw.txt'
    if os.path.exists(pathfile):
        if os.path.exists(pathfile):         os.remove(pathfile)
    toRawCat62(pathfile,dataInicial,dataFinal)
    toCSVCat62(pathfile)


### Pipeline completa

In [10]:
def pipeline_ingestao():
    dataInicial = '2023-03'
    dataFinal = '2023-05'
    print("\n Iniciando Bimtra")
    ingestaoBimtra(dataInicial,dataFinal)
    print("\n Iniciando Cat62")
    ingestaoCat62(dataInicial,dataFinal)
    print("\n Iniciando espera")
    ingestaoEspera(dataInicial,dataFinal)
    print("\n Iniciando Metaf")
    ingestaoMetaf(dataInicial,dataFinal)
    print("\n Iniciando Metar")
    ingestaoMetaR(dataInicial,dataFinal)
    print("\n Iniciando Tc Real")
    ingestaoTcReal(dataInicial,dataFinal)
    print("\n Iniciando Tc Prev")
    ingestaoTcPrev(dataInicial,dataFinal)

pipeline_ingestao()


 Iniciando Bimtra
['2023-03' '2023-04']


  0%|          | 0/1 [00:00<?, ?it/s]


idate: 2023-03-01 , fdate: 2023-04-01


100%|██████████| 1/1 [00:01<00:00,  1.43s/it]

Saved data


...

Saved data

 Iniciando Tc Prev


  0%|          | 0/1 [00:00<?, ?it/s]


idate: 2023-03-01 , fdate: 2023-04-01


100%|██████████| 1/1 [00:00<00:00,  1.23it/s]

Saved data
